Zunächst generieren wir ein paar Daten zur Veranschaulichung

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import calendar

Als Zeitraum nehmen wir den 1.1.2020 bis 1.8.2020, passiert am Wochenende nichts und wir beschränken unseren Datensatz daher auf die Tage Montag-Freitag mit dem Parameter freq="B"

In [99]:
dates = pd.date_range(start="2020-01-01", end="2020-08-03", freq="B")

In [100]:
dates

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-06',
               '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10',
               '2020-01-13', '2020-01-14',
               ...
               '2020-07-21', '2020-07-22', '2020-07-23', '2020-07-24',
               '2020-07-27', '2020-07-28', '2020-07-29', '2020-07-30',
               '2020-07-31', '2020-08-03'],
              dtype='datetime64[ns]', length=154, freq='B')

Unsere Daten erzeugen wir nach gewissen Regeln. Für die Anomalieerkennung spielen diese dann eine wichtige Rolle. Die Regeln sind: <br>
Erwartete Anzahl verarbeiteter Elemente = "elem"
Der Standardfall, den wir als Klasse Normal ("N") bezeichnen wollen:

* elem <= 250 

Allerdings gibt es bestimmte Tage, an denen eine größere Anzahl an Elementen verarbeitet werden sollte. 
Das werden die Large-Klassen L1, L2 und L3:

* L1: An einem unbestimmten Tag in den ersten drei Tagen des Monats,  250 < elem < 2.500
* L2: An einem unbestimmten Tag Ende Januar und Ende Juli (je zwischen 26. und 28. des Monats), 2.500 < elem < 10.000 
* L3: Genau 5-7 Werktage(!) vor Monatsende, Wochenendtage müssen also aus der Differenz abgezogen werden, elem > 10.000



In [101]:
dates.shape

(154,)

Zunächst füllen wir values mit Werten der Klasse N, die anderen Werte werden anschließend zu den bestimmten Tagen die Werte überschreiben

In [102]:
def random_N():
    return np.random.randint(0, 251)

def random_L1():
    return np.random.randint(251, 2501)

def random_L2():
    return np.random.randint(2501, 10001)

def random_L3():
    return np.random.randint(10001, 1e6)

In [103]:
values = np.array([random_N() for i in range(dates.shape[0])])
values.shape

(154,)

In [104]:
values[0:5]

array([174,  71,  22,  70, 230])

In [118]:
numberOfMonths = dates.max().month - dates.min().month + 1

daysWhereL1Happened = [pd.Timestamp("2020-%d-%d" %(i,np.random.randint(1, 4)))  for i in range(1, numberOfMonths + 1)]
daysWhereL1Happened

[Timestamp('2020-01-02 00:00:00'),
 Timestamp('2020-02-01 00:00:00'),
 Timestamp('2020-03-01 00:00:00'),
 Timestamp('2020-04-03 00:00:00'),
 Timestamp('2020-05-03 00:00:00'),
 Timestamp('2020-06-01 00:00:00'),
 Timestamp('2020-07-01 00:00:00'),
 Timestamp('2020-08-03 00:00:00')]

In [119]:
daysWhereL2Happened = [pd.Timestamp("2020-%d-%d" %(i,np.random.randint(26, 29)))for i in [1, 7]]
daysWhereL2Happened

[Timestamp('2020-01-28 00:00:00'), Timestamp('2020-07-27 00:00:00')]

In [120]:
lastDayOfEachMonth = [pd.Timestamp("2020-%d-%d"%(i, calendar.monthrange(2020, i)[1])) for i in range(1, numberOfMonths)]
lastDayOfEachMonth

[Timestamp('2020-01-31 00:00:00'),
 Timestamp('2020-02-29 00:00:00'),
 Timestamp('2020-03-31 00:00:00'),
 Timestamp('2020-04-30 00:00:00'),
 Timestamp('2020-05-31 00:00:00'),
 Timestamp('2020-06-30 00:00:00'),
 Timestamp('2020-07-31 00:00:00')]

In [121]:
sixBusinessDays = pd.offsets.BDay(6)
daysWhereL3Happened = [day - sixBusinessDays for day in lastDayOfEachMonth]
daysWhereL3Happened

[Timestamp('2020-01-23 00:00:00'),
 Timestamp('2020-02-21 00:00:00'),
 Timestamp('2020-03-23 00:00:00'),
 Timestamp('2020-04-22 00:00:00'),
 Timestamp('2020-05-22 00:00:00'),
 Timestamp('2020-06-22 00:00:00'),
 Timestamp('2020-07-23 00:00:00')]

In [122]:
df = pd.DataFrame(columns=["date", "value"])
df["date"] = dates
df["value"] = values
df.head()

,date,value
0,2020-01-01,174
1,2020-01-02,71
2,2020-01-03,22
3,2020-01-06,70
4,2020-01-07,230


In [123]:
for date in daysWhereL1Happened:
    df.at[df.date == date, "value"] = random_L1()

In [124]:
for date in daysWhereL2Happened:
    df.at[df.date == date, "value"] = random_L2()

In [125]:
for date in daysWhereL3Happened:
    df.at[df.date == date, "value"] = random_L3()

Schauen wir uns das Ergebnis einmal an:

In [126]:
df[(df.date.dt.day > 25) & ((df.date.dt.month == 1) | (df.date.dt.month == 7))]

,date,value
18,2020-01-27,189
19,2020-01-28,6863
20,2020-01-29,131
21,2020-01-30,181
22,2020-01-31,235
148,2020-07-27,3474
149,2020-07-28,48
150,2020-07-29,46
151,2020-07-30,219
152,2020-07-31,94


In [127]:
df[df.date.dt.day < 4]

,date,value
0,2020-01-01,174
1,2020-01-02,1646
2,2020-01-03,22
23,2020-02-03,144
43,2020-03-02,9
44,2020-03-03,122
65,2020-04-01,14
66,2020-04-02,29
67,2020-04-03,819
87,2020-05-01,125


In [128]:
df[(df.date.dt.day >= 22) & (df.date.dt.day <= 26)]

,date,value
15,2020-01-22,216
16,2020-01-23,326297
17,2020-01-24,136
38,2020-02-24,61
39,2020-02-25,160
40,2020-02-26,82
58,2020-03-23,985856
59,2020-03-24,149
60,2020-03-25,168
61,2020-03-26,21


Wir konnten also künstlich einen Datensatz erzeugen, der unseren Reglen entspricht. Im Normalfall ist der Weg natürlich anders herum. Wir haben einen Datensatz und müssen die Regeln daraus ableiten. <br>

Wir fügen nun noch ein paar Anomalien hinzu

In [132]:
df.at[147, "value"] = random_L2()
df.at[59, "value"] = random_L1()
df.at[0, "value"] = random_N()

In [133]:
df.iloc[[0, 59, 147]]

,date,value
0,2020-01-01,196
59,2020-03-24,2346
147,2020-07-24,9405


In [134]:
df.to_excel("generatedData.xlsx", index=False)